In [1]:
import numpy as np
import pandas as pd

In [2]:
from openpyxl import load_workbook

In [3]:
from IPython.display import display

In [4]:
import datetime as dt

nsepy package with Total Returns Index history is available at https://github.com/dsanr/nsepy

In [5]:
import nsepy

In [6]:
indexes = ['nifty 50', 'nifty 100', 'nifty 200', 'nifty 500', 'nifty largemidcap 250', 'nifty midcap 50', 'nifty midcap 100', 'nifty midcap 150',
           'nifty smallcap 50', 'nifty smallcap 100', 'nifty smallcap 250']

In [7]:
dt.date.today()

datetime.date(2022, 11, 26)

In [8]:
nsepy.get_total_returns_index_history('nifty 500', dt.date.today()-dt.timedelta(days=3), dt.date.today())

,Total Returns Index
Date,
2022-11-23,24105.44
2022-11-24,24326.89
2022-11-25,24398.91


# Downloading and saving data to new excel file

Uncomment below code to download and save data to new excel file.

In [9]:
# tri = nsepy.get_total_returns_index_history('nifty largemidcap 250', dt.date(2021,1,1), dt.date(2020,12,22))
# tri

In [10]:
# pe = nsepy.get_index_pe_history('nifty largemidcap 250', dt.date(2021,1,1), dt.date(2020,12,22))
# pe

In [11]:
# df = tri.join(pe)
# df.to_excel('nifty largemidcap 250.xlsx')

# Downloading the data for remaining days based on the existing excel file data

In [12]:
def get_data(index):
    '''get TRI history and PE history from nsepy as dataframe'''
    
    #last date in the file
    last = pd.read_excel(index+'.xlsx', index_col=0).index[-1]
    
    # get values of the remaining dates
    tri = nsepy.get_total_returns_index_history(index, last.date()+dt.timedelta(days=1), dt.date.today())
    pe = nsepy.get_index_pe_history(index, last.date()+dt.timedelta(days=1), dt.date.today())
    tri_pe = tri.join(pe)
    return tri_pe

Saving downloaded data to existing excel file

In [13]:
# def save_data(index):
#     '''save the index data to existing excel files'''
    
#     tri_pe = get_data(index)
    
#     existing_data = pd.read_excel(index+'.xlsx', index_col=0)
#     with pd.ExcelWriter(index+'.xlsx', date_format='YYYY-MM-DD') as writer:
#         pd.concat([existing_data, tri_pe]).to_excel(writer)

In [14]:
def save_data(index):
    '''save the index data to existing excel files'''
    
    tri_pe = get_data(index)
    
    wb = load_workbook(index+'.xlsx')
    ws = wb.active
    for i, r in tri_pe.iterrows():
        ws.append([i, r['Total Returns Index'], r['P/E'], r['P/B'], r['Div Yield']])
    wb.save(index+'.xlsx')

In [15]:
for i in indexes:
    save_data(i)

display last two daya of data from excel file

In [16]:
for i in indexes:
    print(i)
    display(pd.read_excel(i+'.xlsx').tail(2))

nifty 50


,Date,Total Returns Index,P/E,P/B,Div Yield
2947,2022-11-24,26877.24,22.21,4.34,1.3
2948,2022-11-25,26918.89,22.25,4.35,1.3


nifty 100


,Date,Total Returns Index,P/E,P/B,Div Yield
2947,2022-11-24,24800.12,23.05,4.47,1.33
2948,2022-11-25,24833.14,23.08,4.48,1.33


nifty 200


...

,Date,Total Returns Index,P/E,P/B,Div Yield
2948,2022-11-24,5313.78,20.27,3.56,1.31
2949,2022-11-25,5375.49,20.50,3.60,1.29


nifty smallcap 100


,Date,Total Returns Index,P/E,P/B,Div Yield
2947,2022-11-24,12335.04,16.72,2.97,1.15
2948,2022-11-25,12473.09,16.91,3.00,1.14


nifty smallcap 250


,Date,Total Returns Index,P/E,P/B,Div Yield
2948,2022-11-24,11806.60,18.69,3.02,1.09
2949,2022-11-25,11898.03,18.84,3.04,1.08


To check whether same date repeats

In [17]:
for i in indexes:
    df=pd.read_excel(i+'.xlsx', index_col=0)
    print(i)
    display(df[df.index.duplicated(keep=False)])
    print(f"total duplicated indexes {df.index.duplicated(keep=False).sum()}\n")

nifty 50


,Total Returns Index,P/E,P/B,Div Yield
Date,,,,


total duplicated indexes 0

nifty 100


,Total Returns Index,P/E,P/B,Div Yield
Date,,,,


total duplicated indexes 0

nifty 200


...

total duplicated indexes 2

nifty smallcap 100


,Total Returns Index,P/E,P/B,Div Yield
Date,,,,


total duplicated indexes 0

nifty smallcap 250


,Total Returns Index,P/E,P/B,Div Yield
Date,,,,
2016-04-01,4742.59,65.64,1.54,1.14
2016-04-01,4742.59,86.86,1.35,1.20


total duplicated indexes 2



Stating date for which P/E values are available

In [18]:
for i in indexes:
    print(i)
    df = pd.read_excel(i+'.xlsx')
    display(df[~df['P/E'].isnull()].head(2))

nifty 50


,Date,Total Returns Index,P/E,P/B,Div Yield
0,2011-01-03,7727.31,24.57,3.88,1.01
1,2011-01-04,7713.22,24.53,3.87,1.01


nifty 100


,Date,Total Returns Index,P/E,P/B,Div Yield
0,2011-01-03,6924.52,23.15,3.76,1.01
1,2011-01-04,6908.94,23.10,3.75,1.02


nifty 200


...

,Date,Total Returns Index,P/E,P/B,Div Yield
1301,2016-04-01,3089.9,24.93,1.20,1.29
1302,2016-04-01,3089.9,27.61,0.99,1.52


nifty smallcap 100


,Date,Total Returns Index,P/E,P/B,Div Yield
1204,2015-11-09,6227.22,22.55,1.02,1.45
1205,2015-11-10,6144.27,22.5,1.00,1.47


nifty smallcap 250


,Date,Total Returns Index,P/E,P/B,Div Yield
1301,2016-04-01,4742.59,65.64,1.54,1.14
1302,2016-04-01,4742.59,86.86,1.35,1.20


In [19]:
nsepy.get_total_returns_index_history('nifty midcap 150', dt.date(2016,4,1), dt.date(2016,4,2))

,Total Returns Index
Date,
2016-04-01,4956.96


In [18]:
nsepy.get_index_pe_history('nifty midcap 150', dt.date(2016,4,1), dt.date(2016,4,2))

,P/E,P/B,Div Yield
Date,,,
2016-04-01,27.45,2.25,1.26
2016-04-01,27.76,2.21,1.20


Examples

In [6]:
sbi = nsepy.get_history('sbin', start=dt.date(2020,8,1), end=dt.date(2020,8,7))
sbi

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2020-08-03,SBIN,EQ,191.45,192.00,194.60,190.05,192.30,192.25,192.68,72457670,1.396079e+15,308823,9698233,0.1338
2020-08-04,SBIN,EQ,192.25,193.35,193.80,190.50,191.95,191.60,191.84,43867110,8.415652e+14,176987,6362150,0.1450
2020-08-05,SBIN,EQ,191.60,192.75,196.85,191.00,191.80,191.45,193.91,68888954,1.335821e+15,271605,9480012,0.1376
2020-08-06,SBIN,EQ,191.45,192.30,194.50,190.25,191.15,190.95,192.34,59740299,1.149069e+15,239032,7633674,0.1278
2020-08-07,SBIN,EQ,190.95,191.45,192.10,189.55,190.85,190.65,190.90,44822757,8.556666e+14,186761,9053179,0.2020


In [9]:
sbi.loc[dt.datetime.strptime('2020-08-03', '%Y-%m-%d').date()]['Open']

192.0

In [17]:
sbi.iloc[[1],[2]]

,Prev Close
Date,
2020-08-04,192.25


In [23]:
sbi.iat[1,2]

192.25